In [1]:
import os
import glob
from tqdm import tqdm
import shutil
from pathlib import Path
from openslide import OpenSlide
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import block_reduce
import cv2.cv2 as cv2

/home/malyshevvalery/.pyenv/versions/3.7.7/envs/imagedl/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
INPUT_DIR = Path('/hdd_barracuda2/PANDA/PANDA_raw')
IMG_DIR = INPUT_DIR / 'train_images'
MASK_DIR = INPUT_DIR / 'train_label_masks'

df = pd.read_csv(INPUT_DIR / 'train.csv')
suspicious = pd.read_csv('/hdd_barracuda2/PANDA/PANDA_Suspicious_Slides.csv')

In [3]:
df = df.loc[~df['image_id'].isin(suspicious['image_id'])]
df.loc[df['gleason_score'] == 'negative','gleason_score'] = '0+0'
df['gleason1'] = df['gleason_score'].str[0].astype(int)
df['gleason2'] = df['gleason_score'].str[2].astype(int)
df.head()

,image_id,data_provider,isup_grade,gleason_score,gleason1,gleason2
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,0,0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,0,0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,4,4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,4,4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,0,0


In [4]:
LEVEL = 0
TILE_SIZE = 256
STEP = 128

OUT_DIR = Path(f'Class_LVL{LEVEL}_{TILE_SIZE}_rad_panda')
OUT_DIR.mkdir(exist_ok=True)

In [5]:
# Rolling 2D window for ND array
def roll(a, sz, step):  # vertical step, ordinate, number of rows
    shape = ((a.shape[0] - sz) // step + 1, 
             (a.shape[1] - sz) // step + 1,
             sz, sz)
    strides = (a.strides[0] * step, a.strides[1] * step) + a.strides[-2:]
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

# Radboud

In [6]:
df_rad = df[df['data_provider'] == 'radboud']
df_rad = df_rad[df_rad['isup_grade'] > 0]
print(df_rad.shape)
OUT_DIR.mkdir(exist_ok = True)

(3584, 6)


In [10]:
counters = np.zeros(5)
for i, row in tqdm(df_rad.iterrows(), total=len(df_rad)):
    image_id = row['image_id']
    slide = OpenSlide(str(IMG_DIR / (image_id + '.tiff')))
    mask = OpenSlide(str(MASK_DIR / (image_id + '_mask.tiff')))
    dim = mask.level_dimensions[LEVEL] # x y
    ds = int(mask.level_downsamples[LEVEL])
    n_tiles = (
        (dim[0] + STEP - TILE_SIZE) // STEP,
        (dim[1] + STEP - TILE_SIZE) // STEP
    )
    
    ts_l = int(TILE_SIZE * ds / mask.level_downsamples[2])
    st_l = int(STEP * ds / mask.level_downsamples[2])
    mask_thumb = np.array(mask.read_region((0,0),2,mask.level_dimensions[2]))[...,0]
    
    excluded = roll(mask_thumb == 0,ts_l, st_l).mean(axis=(2,3))
    rows, cols = np.where(excluded < 0.3)
    for rr,cc in zip(rows, cols):
        x = cc * STEP * ds
        y = rr * STEP * ds
        tile = np.array(slide.read_region((x,y), LEVEL, (TILE_SIZE, TILE_SIZE)))[...,:3]
        mask_tile = np.array(mask.read_region((x,y), LEVEL, (TILE_SIZE, TILE_SIZE)))[...,0]
        white_dist = np.sqrt(((1 - tile / 255) ** 2).sum(2))
        white_score = (white_dist < 0.3).mean()
        if (mask_tile == 0).mean() >= 0.3 or white_score > 0.3:
            continue
        unq, cnt = np.unique(mask_tile, return_counts = True)
        cnt = cnt / TILE_SIZE / TILE_SIZE
        if unq[0] == 0:
            cnt = cnt[1:]
            unq = unq[1:]
        well_presented = unq[cnt > 0.05]
        clazz = np.max(unq) - 1
        counters[clazz] += 1
        cv2.imwrite(str(OUT_DIR / f'{image_id[:8]}_{y}_{x}_{LEVEL}_{clazz}.png'),tile[...,::-1])
#         plt.imshow(tile)
#         plt.title(sum(counters))
#         plt.show()
#         plt.imshow(mask_tile)
#         plt.show()
#         if sum(counters) > 200:
#             break
    slide.close()
    mask.close()
    break

  0%|          | 0/3584 [00:00<?, ?it/s]

[1 2] [0.89949036 0.06134033]
[1 2]
[1 2] [0.96176147 0.03807068]
[1]
[1 2] [0.95898438 0.04101562]
[1]
[1 2] [0.96717834 0.03262329]
[1]
[1 2] [0.96713257 0.03286743]
[1]
[1 2] [0.97335815 0.02412415]
[1]
[1 2 4] [0.86991882 0.00289917 0.12290955]
[1 4]
[1 4] [0.62571716 0.37249756]
[1 4]
[1 4] [0.49977112 0.49993896]
[1 4]
[1 4] [0.49227905 0.45489502]
[1 4]
[1 4] [0.45999146 0.48666382]
[1 4]
[1 4] [0.49806213 0.49916077]
[1 4]
[1 4] [0.43666077 0.56333923]
[1 4]
[1 4] [0.43162537 0.56837463]
[1 4]
[1 4] [0.47038269 0.52961731]
[1 4]
[1 4] [0.56645203 0.43354797]
[1 4]
[1 4] [0.70239258 0.18838501]
[1 4]
[1 4] [0.54127502 0.28684998]
[1 4]
[1 4] [0.70611572 0.29388428]
[1 4]
[1 4] [0.58305359 0.30882263]
[1 4]
[1 4] [0.6618042 0.3381958]
[1 4]
[1 4] [0.46633911 0.50032043]
[1 4]
[1 4] [0.53242493 0.46524048]
[1 4]
[1 4] [0.40765381 0.59135437]
[1 4]
[1 4] [0.503479   0.46124268]
[1 4]
[1 4] [0.39901733 0.60098267]
[1 4]
[1 4] [0.498703   0.40238953]
[1 4]
[1 4] [0.42182922 0.5781707

  0%|          | 0/3584 [00:00<?, ?it/s]


In [ ]:
!tar -czf Class_LVL1_256_rad_panda.tar.gz Class_LVL1_256_rad_panda

In [14]:
!du -h
!du -h Class_LVL1_256_rad_panda.tar.gz

2.3M	./.ipynb_checkpoints
21G	./Class_LVL1_256_rad_panda
122G	./Class_LVL0_256_rad_panda
196K	./ForPres/.ipynb_checkpoints
52M	./ForPres
8.0K	./palette/PANDA/.ipynb_checkpoints
9.2M	./palette/PANDA/selected_4_per_class_kar
6.1M	./palette/PANDA/palette
16M	./palette/PANDA
22M	./palette
273M	./cbir_ds/4
234M	./cbir_ds/45
39M	./cbir_ds/5
33M	./cbir_ds/54
578M	./cbir_ds
265M	./cbir_norm_ds/4
229M	./cbir_norm_ds/45
38M	./cbir_norm_ds/5
32M	./cbir_norm_ds/54
562M	./cbir_norm_ds
273M	./tile_dataset_v2/kar/4
466M	./tile_dataset_v2/kar/3
39M	./tile_dataset_v2/kar/5
658M	./tile_dataset_v2/kar/0
1.5G	./tile_dataset_v2/kar
355M	./tile_dataset_v2/rad/4
357M	./tile_dataset_v2/rad/3
242M	./tile_dataset_v2/rad/5
624M	./tile_dataset_v2/rad/0
1.6G	./tile_dataset_v2/rad
3.0G	./tile_dataset_v2
180M	./5_added
1.1G	./selected
169G	.
21G	Class_LVL1_256_rad_panda.tar.gz


In [12]:
counters

array([ 3215., 31007., 28268., 62233., 23111.])

In [13]:
counters.sum()

147834.0

### Count classes

In [12]:
counter = np.zeros(5)
for f in tqdm(OUT_DIR.iterdir()):
    counter[int(str(f)[-5])] += 1
print(f'{int(counter.sum())} - ' + ', '.join(f'{int(c)} ({int(p*100)}%)' for c,p in zip(counter, counter / counter.sum())))

147834it [00:00, 280232.16it/s]

147834 - 3215 (2%), 31007 (20%), 28268 (19%), 62233 (42%), 23111 (15%)


### Prepare dataframe with info

In [ ]:
all_info = []
filenames = sorted(os.listdir(OUT_DIR))
df_rad['image_id_short'] = df_rad['image_id'].str[:8]

cur_mask = None
cur_short_id = None

for f in tqdm(filenames):
    split = f.split('_')
    id_ = split[0]
    y,x,level = int(split[1]), int(split[2]), int(split[3])
    
    if cur_short_id != id_:
        cur_short_id = id_
        if cur_mask is not None:
            cur_mask.close()        
        row = df_rad.loc[df_rad['image_id_short'] == id_].iloc[0]
        cur_mask = OpenSlide(str(MASK_DIR / (row['image_id'] + '_mask.tiff')))
        
    mask = cur_mask.read_region((x,y), level, (TILE_SIZE, TILE_SIZE))
    mask= np.array(mask)[...,0]
    rl = roll(mask,64,64).max((2,3))[1:3,1:3]
    conf = 1.0 * np.all(rl == rl[0][0])
    unq, cnt = np.unique(mask, return_counts = True)
    cnt = cnt / cnt.sum()
    info = np.zeros(4)
    info[unq[unq > 1] - 2] = cnt[unq > 1]
    info = [f, row['isup_grade'], row['gleason1'], row['gleason2']] + list(info) + [conf]
    all_info.append(info)
cur_mask.close()

  0%|          | 2526/994087 [00:07<51:53, 318.52it/s]

In [ ]:
res_df = pd.DataFrame(columns={'filename':str, 'isup_grade': int, 'gleason1': int, 'gleason2': int, 'epi':float,'3':float,'4':float,'5':float, 'conf':float}, data=all_info)
res_df.head()

In [ ]:
arr = res_df.loc()[:,'epi':].to_numpy()

In [ ]:
for i in range(4):
    iarr = arr[:, i]
    plt.hist(iarr[iarr > 0.01], bins=100,alpha=0.3, label = res_df.columns[1+i])
    szs = [len(iarr), (iarr > 0).sum(), (iarr > 0.005).sum(), (iarr > 0.01).sum(), (iarr > 0.02).sum()]
    print(' '.join(f'{s:.2f}' for s in np.array(szs) / len(iarr)))
plt.legend()
plt.show()

In [ ]:
t = 0.01
counter = (arr > t).sum(0)
print(f'{len(arr)} - ' + ', '.join(f'{int(c)} ({int(np.round(p*100))}%)' for c,p in zip(counter, counter / len(arr))))
print(((arr < t).sum(1) == 4).sum())
print(((arr < t).sum(1) == 4).sum()/ len(arr))

In [ ]:
res_df.to_csv(str(OUT_DIR) + '.csv')

## Add data for norm

In [15]:
to_add = int(counter[1:].mean() - counter[0])
print(to_add)

df_rad = df[df['data_provider'] == 'radboud']
df_rad = df_rad[df_rad['isup_grade'] == 0]
print(df_rad.shape)

per_slide = to_add // len(df_rad)

OUT_DIR = Path(f'Class_LVL{LEVEL}_{TILE_SIZE}_rad_panda_Addon')
OUT_DIR.mkdir(exist_ok=True)

32939
(922, 6)


In [ ]:
counters = np.zeros(5)
for i, row in tqdm(df_rad.iterrows(), total=len(df_rad)):
    image_id = row['image_id']
    slide = OpenSlide(str(IMG_DIR / (image_id + '.tiff')))
    mask = OpenSlide(str(MASK_DIR / (image_id + '_mask.tiff')))
    dim = mask.level_dimensions[LEVEL] # x y
    ds = int(mask.level_downsamples[LEVEL])
    n_tiles = (
        (dim[0] + STEP - TILE_SIZE) // STEP,
        (dim[1] + STEP - TILE_SIZE) // STEP
    )
    
    ts_l = int(TILE_SIZE * ds / mask.level_downsamples[2])
    st_l = int(STEP * ds / mask.level_downsamples[2])
    mask_thumb = np.array(mask.read_region((0,0),2,mask.level_dimensions[2]))[...,0]
    
    excluded = roll(mask_thumb == 0,ts_l, st_l).mean(axis=(2,3))
    rows, cols = np.where(excluded < 0.3)
    ids = list(range(len(rows)))
    np.random.shuffle(ids)
    ids = ids[:per_slide]
    for rr,cc in zip(rows, cols):
        x = cc * STEP * ds
        y = rr * STEP * ds
        tile = np.array(slide.read_region((x,y), LEVEL, (TILE_SIZE, TILE_SIZE)))[...,:3]
        mask_tile = np.array(mask.read_region((x,y), LEVEL, (TILE_SIZE, TILE_SIZE)))[...,0]
        white_dist = np.sqrt(((1 - tile / 255) ** 2).sum(2))
        white_score = (white_dist < 0.3).mean()
        if (mask_tile == 0).mean() >= 0.3 or white_score > 0.3:
            continue
        unq, cnt = np.unique(mask_tile, return_counts = True)
        cnt = cnt / TILE_SIZE / TILE_SIZE
        if unq[0] == 0:
            cnt = cnt[1:]
            unq = unq[1:]
        well_presented = unq[cnt > 0.05]
        clazz = np.max(unq) - 1
        counters[clazz] += 1
#         cv2.imwrite(str(OUT_DIR / f'{image_id[:8]}_{y}_{x}_{LEVEL}_{clazz}.png'),tile[...,::-1])
#         plt.imshow(tile)
#         plt.title(clazz)
#         plt.show()
#         plt.imshow(mask_tile)
#         plt.show()
#         if sum(counters) > 200:
#             break
    slide.close()
    mask.close()

 97%|█████████▋| 897/922 [05:56<00:11,  2.27it/s]

In [26]:
tmp = counters + counter
tmp / tmp.sum()

array([0.02484578, 0.25998363, 0.17794284, 0.39174745, 0.1454803 ])